In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [4]:
driver = webdriver.Chrome()  
url = 'https://www.yogiyo.co.kr/mobile/?utm_source=google&utm_medium=cpc&utm_campaign=sem_bra_orignal&utm_term=%EC%9A%94%EA%B8%B0%EC%9A%94&utm_id=sem_000001&referrer=adjust_tracker%3Dgrm2aa%26adjust_google_network%3Dg%26adjust_google_placement%3D%26adjust_campaign%3Dsem_bra_web_kr_700886066_cpc%26adjust_adgroup%3D38249737818%26adjust_creative%3D%EC%9A%94%EA%B8%B0%EC%9A%94_b&gclid=EAIaIQobChMIv9DaxIL-3AIVA6yWCh0C8glmEAAYASAAEgLbF_D_BwE#/%EC%84%9C%EC%9A%B8/138223/'
driver.get(url) 

In [5]:
# 검색창에 음식점 검색하기
def search_restaurant(restaurant_name):
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[13]/form/div/input').send_keys(restaurant_name)    
    driver.find_element_by_xpath('//*[@id="category_search_button"]').click()    
    
# 검색한 음식점 페이지로 넘어가기    
def go_to_store(storenum):
    driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[2]/div').click()
    
# 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    
# 해당 음식점의 리뷰 갯수 저장
def get_review_count():
    return int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text) 
    
# 페이지 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    
# 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()    

# 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오는중...')
    for _ in trange(click_count):
        click_more_review()
        time.sleep(0.1)
    print('모든 리뷰 불러오기 성공!')
        
# 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    driver.execute_script("window.history.go(-1)")    

In [6]:
def yogiyo_crawling(category):
    go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감 
    print(category+'페이지로 넘어가는 중...')
    time.sleep(2) 
    scroll_bottom()
    
    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                               'Taste','Quantity','Delivery','Date'])
    
    print('Start {} Crawling...'.format(category))
    for i in trange(get_store_count()): # 해당 카테고리의 음식점 개수만큼 돌아감
        try:
            go_to_store(i+2) # 순서대로 각 음식점 페이지로 넘어감
            print(str(i+1)+'번째 음식점 페이지로 넘어가는중...')
            time.sleep(2)
            go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
            print('해당 음식점 리뷰 페이지로 넘어가는중...')
            time.sleep(2)
            stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴
            
            for j in trange(get_review_count()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                try:
                    df.loc[len(df)] = { 
                        'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                        'UserID':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[1]'.format(j+2)).text,
                        'Menu':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[3]'.format(j+2)).text,
                        'Review':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/p'.format(j+2)).text,
                        'Taste':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[3]'.format(j+2)).text,
                        'Quantity':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[6]'.format(j+2)).text,
                        'Delivery':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[2]/div/span[2]/span[9]'.format(j+2)).text,
                        'Date':driver.find_element_by_xpath('//*[@id="review"]/li[{}]/div[1]/span[2]'.format(j+2)).text,
                    }
                except Exception as e:
                    print('리뷰 페이지 에러')
                    print(e)
                    pass
                
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            print('음식점 리스트 페이지로 돌아가는중...')
            time.sleep(2)
            
        except Exception as e:
            print('음식점 페이지 에러')
            print(e)
            pass
                
    print('Finish {} Crawling!!!'.format(category))
    return df

In [7]:
chicken_df = yogiyo_crawling('치킨')

NameError: name 'go_to_category' is not defined

In [160]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기